In [14]:
# Setup Dependencies and Constants

# Dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from api_keys import api_key
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)
random_coordinates = 1500
total_cities = 500

print(f"{str(total_cities)} cities will be found by searching up to {str(random_coordinates)} coordinates\
 between latitude {str(lat_range[0])} and {str(lat_range[1])}, and\
 longitude {str(lng_range[0])} and {str(lng_range[1])}.")
print(f"The cities will be saved in ./{output_data_file}.")

500 cities will be found by searching up to 1500 coordinates between latitude -90 and 90, and longitude -180 and 180.
The cities will be saved in ./output_data/cities.csv.


In [26]:
# Generate Cities List

# Lists for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(low=lat_range[0], high=lat_range[1], size=random_coordinates)
lngs = np.random.uniform(low=lng_range[0], high=lng_range[1], size=random_coordinates)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
cities_found = 0
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)
        cities_found += 1
        if cities_found == 500:
            break

# Print the city count to confirm sufficient count
print(f"{str(len(cities))} cities found")

500 cities found


500